# Valuing Caps and Floors

We value caps and floors using Black's model

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *

####################################################################
#  FINANCEPY BETA Version 0.350 - This build: 30 Apr 2024 at 21:20 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Building a Ibor Curve

In [3]:
value_dt = Date(6, 6, 2018)

In [4]:
spot_days = 0
settle_dt = value_dt
depoDCCType = DayCountTypes.THIRTY_E_360_ISDA

depos = []
depo = IborDeposit(settle_dt, "1M", 0.0230, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "2M", 0.0230, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "3M", 0.0230, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "6M", 0.0230, depoDCCType); depos.append(depo)
depo = IborDeposit(settle_dt, "9M", 0.0230, depoDCCType); depos.append(depo)

In [5]:
accrual = DayCountTypes.THIRTY_E_360
freq = FrequencyTypes.SEMI_ANNUAL
longEnd = DateGenRuleTypes.BACKWARD
swap_type = SwapTypes.PAY

spot_days = 2
settle_dt = value_dt.add_weekdays(spot_days)

swaps = []
swap = IborSwap(settle_dt, "1Y", swap_type, 0.0250, freq, accrual); swaps.append(swap)
swap = IborSwap(settle_dt, "2Y", swap_type, 0.0255, freq, accrual); swaps.append(swap)
swap = IborSwap(settle_dt, "3Y", swap_type, 0.0260, freq, accrual); swaps.append(swap)
swap = IborSwap(settle_dt, "4Y", swap_type, 0.0265, freq, accrual); swaps.append(swap)
swap = IborSwap(settle_dt, "5Y", swap_type, 0.0270, freq, accrual); swaps.append(swap)

In [6]:
libor_curve = IborSingleCurve(value_dt, depos, [], swaps)

## Creating a Cap and Floor

In [7]:
capType = FinCapFloorTypes.CAP
floorType = FinCapFloorTypes.FLOOR

In [8]:
strike_rate = 0.02

In [9]:
cap = IborCapFloor(settle_dt, "2Y", capType, strike_rate)
flr = IborCapFloor(settle_dt, "2Y", floorType, strike_rate)

In [10]:
print(cap)

OBJECT TYPE: IborCapFloor
START DATE: 08-JUN-2018
MATURITY DATE: 08-JUN-2020
STRIKE COUPON: 2.0
OPTION TYPE: FinCapFloorTypes.CAP
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA


In [11]:
print(flr)

OBJECT TYPE: IborCapFloor
START DATE: 08-JUN-2018
MATURITY DATE: 08-JUN-2020
STRIKE COUPON: 2.0
OPTION TYPE: FinCapFloorTypes.FLOOR
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA


# Valuation

### Black's Model

We start with Black's model with 25% volatility

In [12]:
model = Black(0.25)

In [13]:
cap.value(value_dt, libor_curve, model)

11364.821332295927

In [14]:
flr.value(value_dt, libor_curve, model)

830.631774904356

In [15]:
cap.print_leg()

START DATE: 08-JUN-2018
MATURITY DATE: 08-JUN-2020
OPTION TYPE FinCapFloorTypes.CAP
STRIKE (%): 2.0
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE 06-JUN-2018
PAYMENT_dt     YEAR_FRAC   FWD_RATE    INTRINSIC           DF    CAPLET_PV       CUM_PV
    08-JUN-2018          -         -            -     1.000000            -            -
    10-SEP-2018  0.2555556   2.29838       757.99     0.994034       757.99       757.99
    10-DEC-2018  0.2500000   2.28739       710.13     0.988382       760.00      1517.99
    08-MAR-2019  0.2444444   2.29062       698.24     0.982878       815.49      2333.48
    10-JUN-2019  0.2555556   3.08483      2703.56     0.975190      2714.74      5048.22
    09-SEP-2019  0.2472222   2.61121      1464.10     0.968935      1564.70      6612.92
    09-DEC-2019  0.2500000   2.58219      1401.23     0.962721      1550.71      8163.63
    09-MAR-2020  0.2500000   2.57691      1379.61     0.956558      1569.15      97

# Alternative Models

## Shifted Black

Shifted Black gives the same pdf at F+S as Black does at F. So if we want to have the PDF for F=0.25 at -0.25 because rates are negative then you need to set F=-0.50.

In [16]:
model = BlackShifted(0.25, -0.005)

In [17]:
cap.value(value_dt, libor_curve, model)

10931.656739234792

In [18]:
flr.value(value_dt, libor_curve, model)

397.46718184322066

The floor has increased in value as the downside risk is greater.

## SABR Model

In [19]:
alpha = 0.037; beta = 0.5; rho  = 0.1; nu = 0.573

In [20]:
model = SABR(alpha, beta, rho, nu)

In [21]:
cap.value(value_dt, libor_curve, model)

11467.913925057415

In [22]:
flr.value(value_dt, libor_curve, model)

933.7243676658493

## Shifted SABR Model

In [23]:
alpha = 0.037; beta = 0.5; rho  = 0.1; nu = 0.573; shift = -0.005

In [24]:
model = SABRShifted(alpha, beta, rho, nu, shift)

In [25]:
cap.value(value_dt, libor_curve, model)

11834.879375336404

In [26]:
flr.value(value_dt, libor_curve, model)

1300.6898179448326

## Hull White Model

In [27]:
sigma = 0.01; alpha = 0.005

In [28]:
model = HWTree(sigma, alpha)

In [29]:
cap.value(value_dt, libor_curve, model)

14313.311445218462

In [30]:
flr.value(value_dt, libor_curve, model)

2764.9947385541873

Copyright (c) 2020 Dominic O'Kane